In [2]:
#importing the required modules
import os
import time
import pandas as pd

In [8]:
class Flanking:
    def __init__(self, g_start, g_length, mt_start, mt_length):
        self.g_start = int(g_start)
        self.g_length = int(g_length)
        self.mt_start = int(mt_start)
        self.mt_length = int(mt_length)
        self.directory = os.path.join(r'Z:\balint\numt\Michael\data\genome_parts_in_fasta/')
        
    def g_sequence(self, g_id):
        """Get the genomic sequence."""
        g_filename = '%s.fasta' % g_id
        g_filepath = self.directory + g_filename
        with open (g_filepath) as g_file:
            g_seq = g_file.readlines()[1]
        return g_seq
    
    def g_up_flanking(self, g_seq):
        """Get the upstream flanking of the genomic sequence."""
        global g_up_flankings
        flank_start = self.g_start - 200
        flank_end = self.g_start
        if flank_start < 0:
            up_flanking = g_seq[:flank_end]
        else:
            up_flanking = g_seq[flank_start:flank_end]
        g_up_flankings.append(up_flanking)
            
    def g_down_flanking(self, g_seq, g_size):
        """Get the downstream flanking of the genomic sequence."""
        global g_down_flankings
        flank_start = self.g_start + self.g_length
        flank_end = flank_start + 200
        if flank_end > g_size:
            down_flanking = g_seq[flank_start:]
        else:
            down_flanking = g_seq[flank_start:flank_end]
        g_down_flankings.append(down_flanking)
        
    def mt_up_flanking(self, mt_seq, strand):
        """Get the upstream flanking of the mitochondrial sequence."""
        global mt_up_flankings
        if strand == '-':
            flank_end = (len(mt_seq) - self.mt_start) - self.mt_length
            flank_start = flank_end - 200
            if flank_start < 0:
                up_flanking = mt_seq[:flank_end] + mt_seq[flank_start:]
            else:
                up_flanking = mt_seq[flank_start:flank_end]
            mt_up_flankings.append(up_flanking)
        else:
            flank_start = self.mt_start - 200
            flank_end = self.mt_start
            if flank_start < 0:
                up_flanking = mt_seq[:flank_end] + mt_seq[flank_start:]
            else:
                up_flanking = mt_seq[flank_start:flank_end]
            mt_up_flankings.append(up_flanking)
            
    def mt_down_flanking(self, mt_seq, strand):
        """Get the upstream flanking of the mitochondrial sequence."""
        global mt_down_flankings
        if strand == '-':
            flank_start = (len(mt_seq) - self.mt_start)
            flank_end = flank_start + 200
            if flank_end > len(mt_seq):
                circular_correction = (flank_end - flank_start) + len(mt_seq[flank_start:])
                down_flanking = mt_seq[flank_start:] + mt_seq[:circular_correction]
            else:
                down_flanking = mt_seq[flank_start:flank_end]
            mt_down_flankings.append(down_flanking)
        else:
            flank_start = self.mt_start + mt_length
            flank_end = flank_start + 200
            if flank_end > len(mt_seq):
                circular_correction = (flank_end - flank_start) + len(mt_seq[flank_start:])
                down_flanking = mt_seq[flank_start:] + mt_seq[:circular_correction]
            else:
                down_flanking = mt_seq[flank_start:flank_end]
            mt_down_flankings.append(down_flanking)

In [4]:
#reading the previously made table into an array
alignment_df = pd.read_csv(r'Z:\balint\numt\Michael\data/up_lastal_array.csv')

In [5]:
alignment_df.head()

,score,eg2_value,e_value,g_id,g_start,mt_start,g_length,mt_length,g_strand,mt_strand,g_size,g_sequence,mt_sequence
0,17245,0.0,0.0,MT,0,0,17245,17245,+,+,17245,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...
1,3398,0.0,0.0,1,114731432,6976,4680,4681,+,+,194850757,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...
2,3120,0.0,0.0,GL019308,67160,883,4892,4945,+,+,88228,CAAGGCCATGAAGCATGCACACACTGCCCATCGCTCTCCTCAAACA...,CAAGGCCATGAAGCACGCACACACCGCCCGTCACCCTCCTCAAGTG...
3,1888,0.0,0.0,GL019308,62527,12805,3439,3462,+,+,88228,AATTATTCATAGCCTCAATGATGAACAAGACATCCAAAAAACAGGA...,AATTATTCACAGCCTcaaTGATGAACAGGATATCCGAAAGATAGGC...
4,1824,0.0,0.0,AAGW02081594,14016,6061,3040,3073,+,+,24576,TTCTTTGGACACCCTGAGGTATATATTCTTATCCTTCCAGGATTCG...,TTTTTCGGGCACCCCGAAGTATATATTCTTATTCTTCCAGGATTTG...


In [6]:
#get the double mt sequence
mt_seq = ''
with open (r'Z:/balint/numt/Michael/data/two_ch_MT.fasta') as mt_file:
    mt_seq = mt_file.readlines()[1]
len(mt_seq)

34490

In [7]:
g_up_flankings = []
g_down_flankings = []
mt_up_flankings = []
mt_down_flankings = []

In [9]:
#get all the flankings
start = time.time()
for index, row in alignment_df.iterrows():
    g_id = row['g_id']
    g_start = row['g_start']
    g_length = row['g_length']
    g_size = row['g_size']
    mt_start = row['mt_start']
    mt_length = row['mt_length']
    mt_strand = row['mt_strand']
    flanking = Flanking(g_start, g_length, mt_start, mt_length)
    g_sequence = flanking.g_sequence(g_id)
    g_up_flanking = flanking.g_up_flanking(g_sequence)
    g_down_flanking = flanking.g_down_flanking(g_sequence, g_size)
    mt_up_flanking = flanking.mt_up_flanking(mt_seq, mt_strand)
    mt_down_flanking = flanking.mt_down_flanking(mt_seq, mt_strand)
    if index % 20 == 0:
        print('done for %s numts' % index)
end = time.time()
runtime = (end - start) / 60#minutes
runtime

done for 0 numts
done for 20 numts
done for 40 numts
done for 60 numts
done for 80 numts
done for 100 numts
done for 120 numts
done for 140 numts


10.311675834655762

In [10]:
print(len(g_up_flankings))
print(len(g_down_flankings))
print(len(mt_up_flankings))
print(len(mt_down_flankings))

153
153
153
153


In [11]:
print(len(g_up_flankings[0]))
print(len(g_down_flankings[0]))
print(len(mt_up_flankings[0]))
print(len(mt_down_flankings[0]))

0
0
200
200


In [12]:
#append the dataframe with the flanking lists
alignment_df['g_up_flanking'] = g_up_flankings
alignment_df['g_down_flanking'] = g_down_flankings
alignment_df['mt_up_flanking (if strand is negative, the coordinates are corrected)'] = mt_up_flankings
alignment_df['mt_down_flanking (if strand is negative, the coordinates are corrected)'] = mt_down_flankings

In [13]:
alignment_df.head()

,score,eg2_value,e_value,g_id,g_start,mt_start,g_length,mt_length,g_strand,mt_strand,g_size,g_sequence,mt_sequence,g_up_flanking,g_down_flanking,"mt_up_flanking (if strand is negative, the coordinates are corrected)","mt_down_flanking (if strand is negative, the coordinates are corrected)"
0,17245,0.0,0.0,MT,0,0,17245,17245,+,+,17245,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...,,,TACAAATTGTATAATATTTGGACTAAACCCCCTTTCCCACCCCAAG...,GTTAATGTAGCTTAACAACAAAGCAAAGCACTGAAAATGCTTAGAT...
1,3398,0.0,0.0,1,114731432,6976,4680,4681,+,+,194850757,taGTAAAATTATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,TAGTAAAACCATTACATAACTTTGTCGAAGTTAATTTATAGGTTCA...,ACAAAAGACTCTGAATAGCCAAAGCGATCCTGAACAAGAAAAATCA...,TATATATATTTATATATATATGTATATATGTACATATATATAACTA...,CCTTCGCCTCAAAACGAGAAGTAGAGACTATTGAACTAACAACCAC...,AACTGCTAATTCATGCTCCCATGTCTAACCCCATGGCTTTCTCAAC...
2,3120,0.0,0.0,GL019308,67160,883,4892,4945,+,+,88228,CAAGGCCATGAAGCATGCACACACTGCCCATCGCTCTCCTCAAACA...,CAAGGCCATGAAGCACGCACACACCGCCCGTCACCCTCCTCAAGTG...,GATAAACCCCGATAAACTTTACCACTCTTTGCCAACTCAGCCTCTA...,ATGAAGGGGCAAAGAGTAATTTTACAGTCGAGAGGGCTAGCAGTCA...,TACCGCCATCTTCAGCGAACCCTAAAAAGGAGCAAAAGTAAGCTCA...,TAGGGGCTATTAACTTTATTACAACTATTATTAATATGAAAGCCCC...
3,1888,0.0,0.0,GL019308,62527,12805,3439,3462,+,+,88228,AATTATTCATAGCCTCAATGATGAACAAGACATCCAAAAAACAGGA...,AATTATTCACAGCCTcaaTGATGAACAGGATATCCGAAAGATAGGC...,GTAGGAATTATATCCTTCCTCCTCATCGGCTGATGGCATGGCCAAG...,ACGTATACATGAGTTGTTCTTACATAATTGAATATACCTGCAATGC...,TGCCTAGGCGCAATCACTACCTTATTCACAGCCCTATGTGCCCTCA...,GTACACCCGTACGCACGCACGTACACCCGTACGCACGCACGTACAC...
4,1824,0.0,0.0,AAGW02081594,14016,6061,3040,3073,+,+,24576,TTCTTTGGACACCCTGAGGTATATATTCTTATCCTTCCAGGATTCG...,TTTTTCGGGCACCCCGAAGTATATATTCTTATTCTTCCAGGATTTG...,GACATGTCAAAGGAACGCAGAAGCCAGCTTGAAAAAGTCCTCAACA...,TGAAATAGCTATCACTATTCAGGTATGCCACTATTCTTCTAGGCAT...,ATATGAAAGCCCCTGCAATATCTCAATATCAAACCCCCTTATTCGT...,ATTCTCCTAGGTATCTATTTTACCCTACTTCAAGCATCGGAGTATT...


In [16]:
alignment_df.to_csv(r'Z:\balint\numt\Michael\data/numt_master_array.csv', index = False)